In [356]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [357]:
from dataclasses import dataclass

@dataclass
class Entity:
    id: int
    arrival_time: float
    interarrival_time: float
    service_time: float
    # departure_time: float

    def __repr__(self):
        return f"Entity(\
id={self.id}, \
arrival_time={self.arrival_time}, \
interarrival_time={self.interarrival_time}, \
service_time={self.service_time})"
                    # ,departure_time={self.departure_time})"

In [358]:
@dataclass
class SimulationRecord:
    # entity: Entity
    # arrival_time: float
    # interarrival_time: float
    # service_time: float
    # departure_time: float
    # time_in_system: float
    # time_in_queue: float

    # def __repr__(self):
    #     return f"SimulationRecord(\
    pass

In [359]:
class FIFO_queue:
    def __init__(self, q_type, entity_list=None):
        self.q_type = q_type
        if entity_list:
            self.queue = entity_list
        else:
            self.queue = []
    def add(self, entity):
        logger.info(f"[{self.q_type} queue]: ADD {entity}")
        self.queue.insert(0, entity)
    def pop(self):
        pop_entity = self.queue.pop()
        logger.info(f"[{self.q_type} queue]: POP {pop_entity}")
        return pop_entity
    def __repr__(self):
        return f"FIFO_queue({self.queue})"

class Server:
    def __init__(self):
        self.is_busy = False
        # self.busy_until = 0
        self.entity = None
    # def is_free(self, time):
    #     return not self.is_busy or self.busy_until <= time
    # def serve(self, entity, time):
    #     self.is_busy = True
    #     self.busy_until = time + entity.service_time
    #     self.entity = entity
    # def free(self, time):
    #     self.is_busy = False
    #     self.busy_until = 0
    #     self.entity = None
    def __repr__(self):
        return f"Server({self.entity})"

In [360]:
@dataclass
class Status:
    clock: int

    server: Server
    arriving_queue: FIFO_queue
    waiting_queue: FIFO_queue
    finished_queue: FIFO_queue

    B_t: list
    Q_t: list

    num_completed_waiting: int
    total_waiting_time_in_queue: int
    area_under_Q_t: int
    area_under_B_t: int

    unit_in_second: float
    second_pre_unit: int


    def check_waiting_time(self):
        self.Q_t.append((self.clock, len(self.waiting_queue.queue)))
        self.area_under_Q_t += len(self.waiting_queue.queue) * int(self.unit_in_second * self.second_pre_unit)
        self.total_waiting_time_in_queue += len(self.waiting_queue.queue) * int(self.unit_in_second * self.second_pre_unit)

    def check_arrival(self):
        if self.arriving_queue.queue:
            len_arriving_queue = len(self.arriving_queue.queue)
            # logger.debug(f"check_arrival: {self.clock}, len of arriving_queue: {len_arriving_queue}")
            if self.clock == self.arriving_queue.queue[-1].arrival_time:
                logger.info(f"[new_arrival]: clock {self.clock}")
                entity = self.arriving_queue.pop()
                self.waiting_queue.add(entity)

    
    def check_service(self):
        # logger.debug(f"check_service: {self.clock}, server: {self.server}")
        if self.server.entity != None:
            self.B_t.append((self.clock, 1))
            self.area_under_B_t += 1 * int(self.unit_in_second * self.second_pre_unit)
            self.server.entity.service_time -= int(self.unit_in_second * self.second_pre_unit)
            if self.server.entity.service_time == 0:
                self.finished_queue.add(self.server.entity)
                logger.info(f"[server]: FINISH {self.server.entity}, clock {self.clock}")
                self.server.entity = None
                
                # logger.info(f"server: SERVE {self.server.entity}")

        if self.server.entity == None:
            self.B_t.append((self.clock, 0))
            self.area_under_B_t += 0 * int(self.unit_in_second * self.second_pre_unit)
            if len(self.waiting_queue.queue) > 0:
                entity = self.waiting_queue.pop()
                self.num_completed_waiting += 1
                self.server.entity = entity
                logger.info(f"[server]: ADD {self.server.entity}, clock {self.clock}")
    
    def process(self):
        self.check_waiting_time()
        self.check_arrival()
        self.check_service()
        self.clock += int(self.unit_in_second * self.second_pre_unit)
    
    def average_waiting_time_in_queue(self):
        return (self.total_waiting_time_in_queue / self.num_completed_waiting) * self.unit_in_second

    def time_average_number_in_queue(self):
        return self.area_under_Q_t / self.clock

    def utilization(self):
        return self.area_under_B_t / self.clock
        
    def __repr__(self):
        return f"===========\n\
clock: {self.clock}\n\n\
server: {self.server}\n\n\
arriving_queue: {self.arriving_queue}\n\n\
waiting_queue: {self.waiting_queue}\n\n\
finished_queue: {self.finished_queue}\n\n\
area_under_Q_t: {self.area_under_Q_t}, area_under_B_t: {self.area_under_B_t}\n\n\
num_completed_waiting: {self.num_completed_waiting}\n\n\
total_waiting_time_in_queue: {self.total_waiting_time_in_queue}\n\
===========\n"
# B_t: {self.B_t}, Q_t: {self.Q_t}, 
    # def __repr__(self):
    #     return f"Status(\

In [361]:
arrival_time_list = [0.00, 1.73, 3.08, 3.79, 4.41, 18.69, 19.39, 34.91, 38.06, 39.82]
interarrival_time_list = [1.73, 1.35, 0.71, 0.62, 14.28, 0.70, 15.52, 3.15, 1.76, 1.00]
service_time = [2.90, 1.76, 3.39, 4.52, 4.46, 4.36, 2.07, 3.36, 2.37, 5.38]

arrival_time_list = [int(i*100) for i in arrival_time_list]
interarrival_time_list = [int(i*100) for i in interarrival_time_list]
service_time = [int(i*100) for i in service_time]

entity_list = [Entity(i + 1, arrival_time, interarrival_time, service_time) for i, (arrival_time, interarrival_time, service_time) in enumerate(zip(arrival_time_list, interarrival_time_list, service_time))]
entity_list.reverse()

In [362]:
system_status = Status(
    clock = int(0.00*100),
    server = Server(),
    arriving_queue = FIFO_queue("arring", entity_list.copy()),
    waiting_queue = FIFO_queue("waiting", None),
    finished_queue = FIFO_queue("finished", None),
    num_completed_waiting = 0,
    B_t = [], # Server status (time, status)
    Q_t = [], # Queue status (time, num_in_queue)
    total_waiting_time_in_queue = 0,
    area_under_Q_t = 0,
    area_under_B_t = 0,
    unit_in_second = 0.01,
    second_pre_unit = 100,
)
simulation_records = []

In [363]:
system_status

clock: 0

server: Server(None)

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=290)])

waiting_queue: FIFO_queue([])

finished_queue: FIFO_queue([])

area_under_Q_t: 0, area_under_B_t: 0

num_completed_waiting: 0

total_waiting_time_in_queue: 0

In [364]:
while system_status.clock <= int(1.73*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 0
INFO:__main__:[arring queue]: POP Entity(id=1, arrival_time=0, interarrival_time=173, service_time=290)
INFO:__main__:[waiting queue]: ADD Entity(id=1, arrival_time=0, interarrival_time=173, service_time=290)
INFO:__main__:[waiting queue]: POP Entity(id=1, arrival_time=0, interarrival_time=173, service_time=290)
INFO:__main__:[server]: ADD Entity(id=1, arrival_time=0, interarrival_time=173, service_time=290), clock 0
INFO:__main__:[new_arrival]: clock 173
INFO:__main__:[arring queue]: POP Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176)
INFO:__main__:[waiting queue]: ADD Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176)


clock: 174

server: Server(Entity(id=1, arrival_time=0, interarrival_time=173, service_time=117))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)])

waiting_queue: FIFO_queue([Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176)])

finished_queue: FIFO_queue([])

area_under_Q_t: 0, area_under_B_t: 173

num_completed_waiting: 1

total_waiting_time_in_queue: 0

In [365]:
while system_status.clock <= int(2.90*100):
   system_status.process() 
system_status

INFO:__main__:[finished queue]: ADD Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)
INFO:__main__:[server]: FINISH Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0), clock 290
INFO:__main__:[waiting queue]: POP Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176)
INFO:__main__:[server]: ADD Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176), clock 290


clock: 291

server: Server(Entity(id=2, arrival_time=173, interarrival_time=135, service_time=176))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)])

waiting_queue: FIFO_queue([])

finished_queue: FIFO_queue([Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 117, area_under_B_t: 290

num_completed_waiting: 2

total_waiting_time_in_queue: 117

In [366]:
while system_status.clock <= int(3.08*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 308
INFO:__main__:[arring queue]: POP Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)
INFO:__main__:[waiting queue]: ADD Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)


clock: 309

server: Server(Entity(id=2, arrival_time=173, interarrival_time=135, service_time=158))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451)])

waiting_queue: FIFO_queue([Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)])

finished_queue: FIFO_queue([Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 117, area_under_B_t: 308

num_completed_waiting: 2

total_waiting_time_in_queue: 117

In [367]:
while system_status.clock <= int(3.79*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 379
INFO:__main__:[arring queue]: POP Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451)
INFO:__main__:[waiting queue]: ADD Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451)


clock: 380

server: Server(Entity(id=2, arrival_time=173, interarrival_time=135, service_time=87))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446)])

waiting_queue: FIFO_queue([Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)])

finished_queue: FIFO_queue([Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 188, area_under_B_t: 379

num_completed_waiting: 2

total_waiting_time_in_queue: 188

In [368]:
while system_status.clock <= int(4.41*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 441
INFO:__main__:[arring queue]: POP Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446)
INFO:__main__:[waiting queue]: ADD Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446)


clock: 442

server: Server(Entity(id=2, arrival_time=173, interarrival_time=135, service_time=25))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)])

waiting_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)])

finished_queue: FIFO_queue([Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 312, area_under_B_t: 441

num_completed_waiting: 2

total_waiting_time_in_queue: 312

In [369]:
while system_status.clock <= int(4.66*100):
   system_status.process() 
system_status

INFO:__main__:[finished queue]: ADD Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0)
INFO:__main__:[server]: FINISH Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), clock 466
INFO:__main__:[waiting queue]: POP Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339)
INFO:__main__:[server]: ADD Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339), clock 466


clock: 467

server: Server(Entity(id=3, arrival_time=308, interarrival_time=71, service_time=339))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)])

waiting_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451)])

finished_queue: FIFO_queue([Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 387, area_under_B_t: 466

num_completed_waiting: 3

total_waiting_time_in_queue: 387

In [370]:
while system_status.clock <= int(8.05*100):
   system_status.process() 
system_status

INFO:__main__:[finished queue]: ADD Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0)
INFO:__main__:[server]: FINISH Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), clock 805
INFO:__main__:[waiting queue]: POP Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451)
INFO:__main__:[server]: ADD Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451), clock 805


clock: 806

server: Server(Entity(id=4, arrival_time=379, interarrival_time=62, service_time=451))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)])

waiting_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446)])

finished_queue: FIFO_queue([Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1065, area_under_B_t: 805

num_completed_waiting: 4

total_waiting_time_in_queue: 1065

In [371]:
while system_status.clock <= int(12.57*100):
   system_status.process() 
system_status

INFO:__main__:[finished queue]: ADD Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0)
INFO:__main__:[server]: FINISH Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), clock 1256
INFO:__main__:[waiting queue]: POP Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446)
INFO:__main__:[server]: ADD Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=446), clock 1256


clock: 1258

server: Server(Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=445))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)])

waiting_queue: FIFO_queue([])

finished_queue: FIFO_queue([Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1516, area_under_B_t: 1257

num_completed_waiting: 5

total_waiting_time_in_queue: 1516

In [372]:
while system_status.clock <= int(17.03*100):
   system_status.process() 
system_status

INFO:__main__:[finished queue]: ADD Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0)
INFO:__main__:[server]: FINISH Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0), clock 1702


clock: 1704

server: Server(None)

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206), Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)])

waiting_queue: FIFO_queue([])

finished_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1516, area_under_B_t: 1702

num_completed_waiting: 5

total_waiting_time_in_queue: 1516

In [373]:
while system_status.clock <= int(18.69*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 1869
INFO:__main__:[arring queue]: POP Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)
INFO:__main__:[waiting queue]: ADD Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)
INFO:__main__:[waiting queue]: POP Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436)
INFO:__main__:[server]: ADD Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436), clock 1869


clock: 1870

server: Server(Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=436))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336), Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206)])

waiting_queue: FIFO_queue([])

finished_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1516, area_under_B_t: 1702

num_completed_waiting: 6

total_waiting_time_in_queue: 1516

In [374]:
while system_status.clock <= int(19.39*100):
   system_status.process() 
system_status

INFO:__main__:[new_arrival]: clock 1939
INFO:__main__:[arring queue]: POP Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206)
INFO:__main__:[waiting queue]: ADD Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206)


clock: 1940

server: Server(Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=366))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336)])

waiting_queue: FIFO_queue([Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206)])

finished_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1516, area_under_B_t: 1772

num_completed_waiting: 6

total_waiting_time_in_queue: 1516

In [375]:
while system_status.clock <= int(20.00*100):
   system_status.process() 
system_status

clock: 2001

server: Server(Entity(id=6, arrival_time=1869, interarrival_time=70, service_time=305))

arriving_queue: FIFO_queue([Entity(id=10, arrival_time=3982, interarrival_time=100, service_time=538), Entity(id=9, arrival_time=3806, interarrival_time=176, service_time=237), Entity(id=8, arrival_time=3490, interarrival_time=315, service_time=336)])

waiting_queue: FIFO_queue([Entity(id=7, arrival_time=1939, interarrival_time=1552, service_time=206)])

finished_queue: FIFO_queue([Entity(id=5, arrival_time=441, interarrival_time=1428, service_time=0), Entity(id=4, arrival_time=379, interarrival_time=62, service_time=0), Entity(id=3, arrival_time=308, interarrival_time=71, service_time=0), Entity(id=2, arrival_time=173, interarrival_time=135, service_time=0), Entity(id=1, arrival_time=0, interarrival_time=173, service_time=0)])

area_under_Q_t: 1577, area_under_B_t: 1833

num_completed_waiting: 6

total_waiting_time_in_queue: 1577

In [376]:
#! TODO
print(f"average waiting time in queue {system_status.average_waiting_time_in_queue()}")
#! TODO
print(f"time average number in queue {system_status.time_average_number_in_queue()}")
#! TODO
print(f"utilization {system_status.utilization()}")


average waiting time in queue 2.6283333333333334
time average number in queue 0.7881059470264867
utilization 0.9160419790104948
